In [88]:
import pandas as pd
import numpy as np
import json
import re

In [89]:
# Dataset attained from: https://www.cs.cornell.edu/~arb/data/genius-expertise/

# Lyrics Dataset

In [90]:
with open('./data/genius-expertise/lyrics.jl', encoding='utf-8') as f:
    temp = []
    for line in f.readlines():
        temp.append(json.loads(line))
lyrics_df = pd.DataFrame(temp)

In [91]:
# looking at an example
lyrics_df.iloc[0]['lyrics']

"\n\n[Produced by T-Minus]\n\n[Intro]\nPour up (Drank), head shot (Drank)\nSit down (Drank), stand up (Drank)\nPass out (Drank), wake up (Drank)\nFaded (Drank), faded (Drank)\n\n[Verse 1]\nNow I done grew up 'round some people livin' their life in bottles\nGranddaddy had the golden flask\nBackstroke every day in Chicago\nSome people like the way it feels\nSome people wanna kill their sorrows\nSome people wanna fit in with the popular, that was my problem\nI was in a dark room, loud tunes\nLookin' to make a vow soon\nThat I'ma get fucked up, fillin' up my cup I see the crowd mood\nChangin' by the minute and the record on repeat\nTook a sip, then another sip, then somebody said to me\n\n[Chorus]\nNigga, why you babysittin' only two or three shots?\nI'ma show you how to turn it up a notch\nFirst you get a swimming pool full of liquor, then you dive in it\nPool full of liquor, then you dive in it\nI wave a few bottles, then I watch 'em all flock\nAll the girls wanna play Baywatch\nI got a 

In [92]:
# looks like not much is required other than removing the text beteween brackets

re.sub("\[.*?\]", "", lyrics_df.iloc[0]['lyrics']).strip('\n')

"Pour up (Drank), head shot (Drank)\nSit down (Drank), stand up (Drank)\nPass out (Drank), wake up (Drank)\nFaded (Drank), faded (Drank)\n\n\nNow I done grew up 'round some people livin' their life in bottles\nGranddaddy had the golden flask\nBackstroke every day in Chicago\nSome people like the way it feels\nSome people wanna kill their sorrows\nSome people wanna fit in with the popular, that was my problem\nI was in a dark room, loud tunes\nLookin' to make a vow soon\nThat I'ma get fucked up, fillin' up my cup I see the crowd mood\nChangin' by the minute and the record on repeat\nTook a sip, then another sip, then somebody said to me\n\n\nNigga, why you babysittin' only two or three shots?\nI'ma show you how to turn it up a notch\nFirst you get a swimming pool full of liquor, then you dive in it\nPool full of liquor, then you dive in it\nI wave a few bottles, then I watch 'em all flock\nAll the girls wanna play Baywatch\nI got a swimming pool full of liquor and they dive in it\nPool 

In [93]:
# map function to entire column

lyrics_df['lyrics'] = lyrics_df['lyrics'].apply(lambda x: re.sub("\n+", "\n", re.sub("\[.*?\]", "", x)).strip('\n'))
lyrics_df

,song,lyrics
0,Kendrick-lamar-swimming-pools-drank-lyrics,"Pour up (Drank), head shot (Drank)\nSit down (..."
1,Kendrick-lamar-money-trees-lyrics,"Uh, me and my niggas tryna get it, ya bish (ya..."
2,Kendrick-lamar-xxx-lyrics,"America, God bless you if it's good to you\nAm..."
3,A-ap-rocky-fuckin-problems-lyrics,"I love bad bitches, that's my fuckin' problem\..."
4,Kendrick-lamar-dna-lyrics,"I got, I got, I got, I got—\nLoyalty, got roya..."
...,...,...
37988,Pnl-tchiki-tchiki-lyrics,"Ouais, ouais, ouais, ouais, ouais\nJ'sens l'od..."
37989,Pnl-chang-lyrics,"Chang, chang, chang, j'marche sur la muraille ..."
37990,Pnl-simba-lyrics,"Ouais, Ah on va voir\nOuais, ouais\nEt ça r'co..."
37991,Pnl-je-thaine-version-orange-lyrics,Que dire à part que ça pue dans la street à pa...


In [94]:
# cleaning song column

songs_dict = {}
with open('./data/genius-expertise/artist_info.json', encoding='utf-8') as f:
    temp = []
    for line in f.readlines():
        line = json.loads(line)
        for song in line['songs']:
            if '-and-' in song or 'mtv' in song or 'version' in song:
                continue
            artist = line['url_name']
            if artist in song:
                if song in songs_dict:
                    pass
                else:
                    clean = ' '.join(song.replace(artist, ' ').split('-')[1:-1])
                    if clean:
                        songs_dict[song] = clean


In [95]:
lyrics_df['song'] = lyrics_df['song'].apply(lambda x: songs_dict.get(x, np.nan))
lyrics_df['lyrics'] = lyrics_df['lyrics'].apply(lambda x: x if x else np.nan)
lyrics_df = lyrics_df.dropna()
lyrics_df.columns = ['title', 'lyrics']

In [96]:
# save cleaned dataset
lyrics_df.to_csv('./data/genius-expertiste_clean/songs.csv', index=False)

# Uploading Dataset to Hugging Face

In [97]:
from huggingface_hub import notebook_login
notebook_login()

In [98]:
from datasets import load_dataset

songs = load_dataset('csv', data_files='./data/genius-expertiste_clean/songs.csv')

# create train, validation, test splits (80/10/10)

from datasets import DatasetDict

songs = songs['train'].train_test_split(test_size=0.2)
songs_validation_test = songs['test'].train_test_split(test_size=0.5)

songs = DatasetDict({
    'train': songs['train'],
    'test': songs_validation_test['train'],
    'validation': songs_validation_test['test'],
    
})

songs.push_to_hub("miscjose/genius")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/28 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

Deleting unused files from dataset repository:   0%|          | 0/1 [00:00<?, ?it/s]